In [ ]:
# (Optional) GPU + Torch sanity
!nvidia-smi -L || echo "No GPU found"
import torch, platform
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available(), "| Python:", platform.python_version())

# Mount Google Drive (your dataset lives here)
from google.colab import drive
drive.mount('/content/drive')
DATASET_ROOT = "/content/drive/MyDrive/DLRV/DATASET"  # <-- your path
print("DATASET_ROOT:", DATASET_ROOT)


GPU 0: Tesla T4 (UUID: GPU-6fe50f88-2376-59df-c7bd-100ef6d1b6e3)
Torch: 2.8.0+cu126 | CUDA available: True | Python: 3.12.11
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DATASET_ROOT: /content/drive/MyDrive/DLRV/DATASET


In [ ]:
%cd /content
!rm -rf yolov7
!git clone https://github.com/WongKinYiu/yolov7 -q
%cd yolov7
!python -c "import pathlib; import inspect, train, test, detect; print('OK:', pathlib.Path(inspect.getfile(train)).parent)"


/content
/content/yolov7
2025-09-07 21:47:52.206624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757281672.226317    1472 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757281672.232105    1472 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757281672.246781    1472 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757281672.246812    1472 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757281672.246816    1472 computation_placer.cc:17

In [ ]:
CLASSIF_DIR = "/content/drive/MyDrive/DLRV/DATASET"  # has TRAIN/ and TEST/
YOLO_OUT    = "/content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN"  # new dataset
VAL_SPLIT   = 0.20  # from TRAIN -> train/val


In [ ]:
import os, shutil, io, numpy as np, pandas as pd
from pathlib import Path
from PIL import Image
import cv2

CLASSIF = Path(CLASSIF_DIR)
QUAR    = CLASSIF / "bad"
IMG_EXT = {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}
MIN_WH  = 20

def is_ok_image(p: Path):
    try:
        with Image.open(p) as im:
            im.verify()            # header/structure
        with Image.open(p) as im2:
            im2.load()             # full decode
            w, h = im2.size
        if w < MIN_WH or h < MIN_WH:
            return False, f"too_small_{w}x{h}"
    except Exception as e:
        return False, f"pillow_{type(e).__name__}"
    # extra cv2 decode check
    try:
        data = np.fromfile(str(p), dtype=np.uint8)
        img = cv2.imdecode(data, cv2.IMREAD_COLOR)
        if img is None: return False, "cv2_imdecode_failed"
    except Exception:
        return False, "cv2_exception"
    return True, "ok"

rows = []
for split in ["TRAIN","TEST"]:
    for img in (CLASSIF/split).rglob("*"):
        if not img.is_file() or img.suffix.lower() not in IMG_EXT:
            continue
        ok, reason = is_ok_image(img)
        lbl = None  # (no labels yet at this stage)
        rows.append({"split":split, "image":str(img), "status":"keep" if ok else "quarantine", "reason":reason})
        if not ok:
            qdst = QUAR / split / img.name
            qdst.parent.mkdir(parents=True, exist_ok=True)
            shutil.move(str(img), str(qdst))

report = pd.DataFrame(rows)
report_path = CLASSIF / "clean_report.csv"
report.to_csv(report_path, index=False)
print("Saved:", report_path)
report["status"].value_counts(dropna=False)


Saved: /content/drive/MyDrive/DLRV/DATASET/clean_report.csv


,count
status,
keep,1550
quarantine,1


In [ ]:
import random, math
from pathlib import Path
from PIL import Image

YOLO = Path(YOLO_OUT)
for p in [YOLO/"images/train", YOLO/"labels/train", YOLO/"images/val", YOLO/"labels/val", YOLO/"images/test", YOLO/"labels/test"]:
    p.mkdir(parents=True, exist_ok=True)

def list_images(folder):
    return [p for p in Path(folder).rglob("*") if p.suffix.lower() in IMG_EXT]

classes = sorted([d.name for d in (CLASSIF/"TRAIN").iterdir() if d.is_dir()])
cls2id  = {c:i for i,c in enumerate(classes)}

def write_fullbox(lbl_path: Path, cid: int):
    lbl_path.write_text(f"{cid} 0.5 0.5 1.0 1.0\n", encoding="utf-8")

# TRAIN -> train/val; TEST -> test
import shutil
random.seed(42)
for split_in, split_out in [("TRAIN","train"), ("TEST","test")]:
    for cls in classes:
        imgs = list_images(CLASSIF/split_in/cls)
        if split_out == "train":
            random.shuffle(imgs)
            k = int(math.floor(len(imgs)*(1-VAL_SPLIT)))
            parts = [("train", imgs[:k]), ("val", imgs[k:])]
        else:
            parts = [("test", imgs)]
        for out, files in parts:
            for src in files:
                # only proceed if file still exists (not quarantined)
                if not src.exists():
                    continue
                # ensure readable RGB (already checked, but double-safe)
                try:
                    Image.open(src).convert("RGB")
                except Exception:
                    continue
                dst_img = YOLO/f"images/{out}/{cls}__{src.stem}{src.suffix.lower()}"
                dst_lbl = YOLO/f"labels/{out}/{cls}__{src.stem}.txt"
                shutil.copy2(src, dst_img)
                write_fullbox(dst_lbl, cls2id[cls])

print("Classes:", classes)
print("YOLO dataset root:", YOLO)


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Classes: ['downdog', 'goddess', 'plank', 'tree', 'warrior2']
YOLO dataset root: /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN


In [ ]:
import yaml
DATA_YAML = "/content/yolov7/data/yoga_raw_clean.yaml"
cfg = {
    "train": str((YOLO/"images/train").resolve()),
    "val":   str((YOLO/"images/val").resolve()),
    "test":  str((YOLO/"images/test").resolve()) if (YOLO/"images/test").exists() else "",
    "nc": len(classes),
    "names": classes
}
with open(DATA_YAML, "w", encoding="utf-8") as f:
    yaml.safe_dump(cfg, f, sort_keys=False)
print(open(DATA_YAML).read())


train: /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/images/train
val: /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/images/val
test: /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/images/test
nc: 5
names:
- downdog
- goddess
- plank
- tree
- warrior2



In [ ]:
# from inside your current Colab runtime
%cd /content
!rm -rf yolov7
!git clone https://github.com/WongKinYiu/yolov7 -q
%cd yolov7
!git checkout main -q


/content
/content/yolov7


In [ ]:
# This file is auto-imported by Python on start (module: site -> sitecustomize)
%cd /content/yolov7
code = r"""
# Auto-loaded before train.py runs.
# Allowlist YOLOv7's model class so torch.load(weights_only=True) can unpickle it safely.
try:
    import models.yolo as yolo
    import torch.serialization as ts
    ts.add_safe_globals([yolo.Model])
except Exception:
    # If anything goes wrong, don't block startup.
    pass
"""
open("sitecustomize.py","w", encoding="utf-8").write(code)
print("Wrote /content/yolov7/sitecustomize.py")


/content/yolov7
Wrote /content/yolov7/sitecustomize.py


In [ ]:
%cd /content/yolov7
!wget -q https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt -O yolov7-tiny.pt
RUN_NAME = "y7_tiny_raw_clean"
!python train.py --workers 8 --device 0 --batch-size 16 --img 640 --epochs 50 \
  --data {DATA_YAML} --weights yolov7-tiny.pt --name {RUN_NAME}


/content/yolov7
2025-09-07 22:25:31.969934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757283931.989890   13255 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757283931.995888   13255 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757283932.011524   13255 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757283932.011550   13255 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757283932.011554   13255 computation_placer.cc:177] comput

In [ ]:
from pathlib import Path

# Your classification dataset (has TRAIN/ and TEST/)
CLASSIF_DIR = Path("/content/drive/MyDrive/DLRV/DATASET")

# The YOLO-converted, cleaned dataset you trained on earlier
YOLO_DATA_ROOT = Path("/content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN")

assert CLASSIF_DIR.exists(), f"Missing: {CLASSIF_DIR}"
assert YOLO_DATA_ROOT.exists(), f"Missing: {YOLO_DATA_ROOT} (run the conversion/clean step first)"

# Derive class names from the class folders under TRAIN/
classes = sorted([d.name for d in (CLASSIF_DIR/"TRAIN").iterdir() if d.is_dir()])
print("Classes:", classes)
print("YOLO images:", (YOLO_DATA_ROOT/'images').exists(), "labels:", (YOLO_DATA_ROOT/'labels').exists())


Classes: ['downdog', 'goddess', 'plank', 'tree', 'warrior2']
YOLO images: True labels: True


In [ ]:
import yaml, os

DATA_YAML = "/content/yolov7/data/yoga_raw_clean.yaml"
cfg = {
    "train": str((YOLO_DATA_ROOT/"images/train").resolve()),
    "val":   str((YOLO_DATA_ROOT/"images/val").resolve()),
    "test":  str((YOLO_DATA_ROOT/"images/test").resolve()) if (YOLO_DATA_ROOT/"images/test").exists() else "",
    "nc": len(classes),
    "names": classes
}

os.makedirs("/content/yolov7/data", exist_ok=True)
with open(DATA_YAML, "w", encoding="utf-8") as f:
    yaml.safe_dump(cfg, f, sort_keys=False)

print("Wrote YAML to:", DATA_YAML)
print(open(DATA_YAML, "r", encoding="utf-8").read())


Wrote YAML to: /content/yolov7/data/yoga_raw_clean.yaml
train: /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/images/train
val: /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/images/val
test: /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/images/test
nc: 5
names:
- downdog
- goddess
- plank
- tree
- warrior2



In [ ]:
from pathlib import Path
for k in ("train","val","test"):
    p = cfg[k]
    if p:
        print(k, "ok:", Path(p).exists(), "->", p)


train ok: True -> /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/images/train
val ok: True -> /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/images/val
test ok: True -> /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/images/test


In [ ]:
from pathlib import Path

YOLO_DATA_ROOT = Path("/content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN")  # <- the one used in your YAML
assert (YOLO_DATA_ROOT/"images/train").exists(), "images/train missing"
assert (YOLO_DATA_ROOT/"labels/train").exists(), "labels/train missing"
print("Cleaning:", YOLO_DATA_ROOT)


Cleaning: /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN


In [ ]:
import os, io, shutil, numpy as np, pandas as pd
from pathlib import Path
from PIL import Image, ImageFile
import cv2

IMG_EXTS = {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}
MIN_WH = 20
QUAR   = YOLO_DATA_ROOT/"bad"

def has_jpeg_eoi(p: Path):
    if p.suffix.lower() not in {".jpg",".jpeg"}: return True
    try:
        with open(p, "rb") as f:
            f.seek(-2, os.SEEK_END)
            return f.read() == b"\xff\xd9"
    except Exception:
        return False

def pillow_verify_and_load(p: Path):
    try:
        with Image.open(p) as im:
            im.verify()
        with Image.open(p) as im2:
            im2.load()
            w,h = im2.size
        if w < MIN_WH or h < MIN_WH:
            return False, f"too_small_{w}x{h}"
        return True, "ok"
    except Exception as e:
        return False, f"pillow_{type(e).__name__}"

def cv2_can_read(p: Path):
    try:
        buf = np.fromfile(str(p), dtype=np.uint8)
        img = cv2.imdecode(buf, cv2.IMREAD_COLOR)
        return img is not None
    except Exception:
        return False

def try_reencode(src: Path) -> bool:
    """Try to salvage by loading with truncated allowed, then re-encoding to a clean JPEG."""
    try:
        # allow truncated just for the read attempt
        old = ImageFile.LOAD_TRUNCATED_IMAGES
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        with Image.open(src) as im:
            im = im.convert("RGB")
            tmp = src.with_suffix(".fixing.jpg")
            im.save(tmp, "JPEG", quality=95, optimize=True)
        ImageFile.LOAD_TRUNCATED_IMAGES = old
        # verify the new file strictly
        ok, _ = pillow_verify_and_load(tmp)
        if ok and has_jpeg_eoi(tmp):
            shutil.move(str(tmp), str(src))
            return True
        tmp.unlink(missing_ok=True)
    except Exception:
        pass
    finally:
        try: ImageFile.LOAD_TRUNCATED_IMAGES = old
        except: pass
    return False

def move_with_label(img_path: Path, split: str):
    lbl_src = (YOLO_DATA_ROOT/"labels"/split/img_path.name).with_suffix(".txt")
    dst_img = QUAR/"images"/split/img_path.name
    dst_lbl = QUAR/"labels"/split/lbl_src.name
    dst_img.parent.mkdir(parents=True, exist_ok=True)
    dst_lbl.parent.mkdir(parents=True, exist_ok=True)
    shutil.move(str(img_path), str(dst_img))
    if lbl_src.exists():
        shutil.move(str(lbl_src), str(dst_lbl))

def clean_split(split: str):
    rows = []
    img_dir = YOLO_DATA_ROOT/"images"/split
    if not img_dir.exists(): return rows
    for p in img_dir.rglob("*"):
        if not p.is_file() or p.suffix.lower() not in IMG_EXTS:
            continue

        ok1, r1 = pillow_verify_and_load(p)
        ok2 = has_jpeg_eoi(p)
        ok3 = cv2_can_read(p)
        ok = ok1 and ok2 and ok3

        status = "keep"
        action = "none"
        reason = "ok" if ok else "|".join([r1 if not ok1 else "",
                                           "bad_eoi" if not ok2 else "",
                                           "cv2_fail" if not ok3 else ""]).strip("|")

        if not ok:
            # try repair once
            if p.suffix.lower() in {".jpg",".jpeg"} and try_reencode(p):
                status, action, reason = "keep", "reencoded", reason+"->fixed"
            else:
                status, action = "quarantine", "moved"
                move_with_label(p, split)

        rows.append({"split":split,"image":str(p),"status":status,"action":action,"reason":reason})
    return rows

all_rows = []
for split in ["train","val","test"]:
    print("Scanning:", split)
    all_rows += clean_split(split)

df = pd.DataFrame(all_rows)
rep = YOLO_DATA_ROOT/"clean_fix_report.csv"
df.to_csv(rep, index=False)
print("Saved report:", rep)
df["status"].value_counts(dropna=False)


Scanning: train
Scanning: val
Scanning: test
Saved report: /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/clean_fix_report.csv


,count
status,
keep,1550


In [ ]:
# YOLO caches label/index info under labels/*.cache; delete to refresh after cleaning
deleted = []
for p in (YOLO_DATA_ROOT/"labels").rglob("*.cache"):
    deleted.append(str(p))
    p.unlink(missing_ok=True)
print("Deleted caches:", len(deleted))


Deleted caches: 2


In [ ]:
%cd /content/yolov7
RUN_NAME = "y7_tiny_raw_clean"
!python train.py --workers 8 --device 0 --batch-size 16 --img 640 --epochs 50 \
  --data /content/yolov7/data/yoga_raw_clean.yaml --weights yolov7-tiny.pt --name {RUN_NAME}


/content/yolov7
2025-09-07 22:43:15.426815: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757284995.447827   17953 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757284995.454064   17953 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757284995.469709   17953 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757284995.469734   17953 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757284995.469738   17953 computation_placer.cc:177] comput

In [ ]:
from pathlib import Path
import yaml

RUN_NAME = "y7_tiny_raw_clean3"  # <- your actual run folder from the log
DATA_YAML = "/content/yolov7/data/yoga_raw_clean.yaml"

RUN_WEIGHTS = Path(f"runs/train/{RUN_NAME}/weights")
WEIGHTS = RUN_WEIGHTS/"best.pt" if (RUN_WEIGHTS/"best.pt").exists() else RUN_WEIGHTS/"last.pt"
assert WEIGHTS.exists(), f"No weights found under {RUN_WEIGHTS}. Did training finish writing files?"

with open(DATA_YAML, "r") as f:
    ds = yaml.safe_load(f)

TEST_IMAGES_DIR = Path(ds.get("test") or ds["val"])  # fallback to val if no test
print("Using weights:", WEIGHTS)
print("Test images dir:", TEST_IMAGES_DIR)


Using weights: runs/train/y7_tiny_raw_clean3/weights/best.pt
Test images dir: /content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/images/test


In [ ]:
%cd /content/yolov7
import sys, runpy, importlib
import numpy as np
import torch.serialization as ts

# 1) YOLO/torch classes you already added (safe to re-add)
from models.yolo import Model, Detect
from models.common import Conv, Concat, MP, SP
from torch.nn.modules import batchnorm, pooling, upsampling, container, activation, conv

ts.add_safe_globals([
    Model, Detect, Conv, Concat, MP, SP,
    batchnorm.BatchNorm2d, pooling.MaxPool2d, upsampling.Upsample,
    container.Sequential, container.ModuleList, activation.LeakyReLU, conv.Conv2d
])

# 2) Add NumPy core types that often appear in checkpoints
ts.add_safe_globals([
    np.ndarray, np.dtype, np.number, np.integer, np.floating, np.bool_,
    np.int8, np.int16, np.int32, np.int64, np.float16, np.float32, np.float64,
    # These two are the ones your error mentioned explicitly / commonly needed:
    np.dtypes.Float64DType, np.dtypes.Int64DType,
])

# 3) Old pickling helpers sometimes used in legacy ckpts
try:
    from numpy._core.multiarray import _reconstruct
    ts.add_safe_globals([_reconstruct])
except Exception:
    pass

# 4) (Optional) introspect and allow-list anything else the ckpt asks for
unsafe = ts.get_unsafe_globals_in_checkpoint(str(WEIGHTS))
print("Still-unsafe before run:", [u for u in unsafe if u not in {
    'numpy.ndarray','numpy.dtype','numpy._core.multiarray._reconstruct',
    'numpy._core.multiarray.scalar','numpy.dtypes.Float64DType','numpy.dtypes.Int64DType'
}])

# 5) Hand off to detect.py exactly like CLI
sys.argv = [
    "detect.py",
    "--weights", str(WEIGHTS),
    "--img", "640",
    "--conf", "0.001",
    "--source", str(TEST_IMAGES_DIR),
    "--save-txt", "--save-conf",
    "--name", f"{RUN_NAME}_test"
]
runpy.run_module("detect", run_name="__main__")


/content/yolov7
Still-unsafe before run: []
Namespace(weights=['runs/train/y7_tiny_raw_clean3/weights/best.pt'], source='/content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/images/test', img_size=640, conf_thres=0.001, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=True, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='y7_tiny_raw_clean3_test', exist_ok=False, no_trace=False)
Fusing layers... 
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



/usr/local/lib/python3.12/dist-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


3 downdogs, 2 goddesss, 3 planks, 5 trees, 1 warrior2, Done. (4.4ms) Inference, (58.5ms) NMS
 The image with the result is saved in: runs/detect/y7_tiny_raw_clean3_test2/downdog__00000000.jpg
3 downdogs, 1 goddess, 3 planks, 4 trees, 2 warrior2s, Done. (4.6ms) Inference, (1.2ms) NMS
 The image with the result is saved in: runs/detect/y7_tiny_raw_clean3_test2/downdog__00000001.jpg
3 downdogs, 4 goddesss, 3 planks, 4 trees, 2 warrior2s, Done. (4.8ms) Inference, (1.2ms) NMS
 The image with the result is saved in: runs/detect/y7_tiny_raw_clean3_test2/downdog__00000002.jpg
2 downdogs, 2 goddesss, 5 planks, 5 trees, 1 warrior2, Done. (4.7ms) Inference, (1.2ms) NMS
 The image with the result is saved in: runs/detect/y7_tiny_raw_clean3_test2/downdog__00000003.jpg
3 downdogs, 1 goddess, 3 planks, 5 trees, 1 warrior2, Done. (9.5ms) Inference, (2.4ms) NMS
 The image with the result is saved in: runs/detect/y7_tiny_raw_clean3_test2/downdog__00000004.jpg
3 downdogs, 1 goddess, 4 planks, 2 trees, 1 

{'__name__': '__main__',
 '__file__': '/content/yolov7/detect.py',
 '__cached__': '/content/yolov7/__pycache__/detect.cpython-312.pyc',
 '__doc__': None,
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x7d048f643ce0>,
 '__package__': '',
 '__spec__': ModuleSpec(name='detect', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7d048f643ce0>, origin='/content/yolov7/detect.py'),
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifiers of Python; for example, builtins.len is\nthe full name for the built-in function len().\n\nThis module is not normally accessed explicitly by most\napplications, but can be useful in modules that provide\nobjects with the same name as a built-in value, but in\nwhich the built-in of that name is also needed.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

CLASS_NAMES = ds["names"]; NUM_CLASSES = int(ds["nc"])
TEST_LABELS_DIR = Path(str(TEST_IMAGES_DIR).replace("/images/","/labels/"))
PRED_DIR = Path(f"runs/detect/{RUN_NAME}_test/labels")

# Ground truth per image (first class in .txt)
IMG_EXTS = {".jpg",".jpeg",".png",".bmp"}
gt = {}
for img in TEST_IMAGES_DIR.rglob("*"):
    if img.suffix.lower() in IMG_EXTS:
        lab = (TEST_LABELS_DIR / img.name).with_suffix(".txt")
        if lab.exists():
            parts = lab.read_text().strip().split()
            if parts:
                gt[str(img.resolve())] = int(parts[0])

# Predictions: top-confidence class per image (if any)
pred = {}
if PRED_DIR.exists():
    for txt in PRED_DIR.rglob("*.txt"):
        lines = [ln.strip().split() for ln in txt.read_text().splitlines() if ln.strip()]
        if not lines: continue
        pairs = [(int(p[0]), float(p[-1]) if len(p)>=6 else 0.0) for p in lines]
        best_cls, _ = max(pairs, key=lambda t: t[1])
        stem = txt.stem
        cands = list(TEST_IMAGES_DIR.rglob(stem + ".*"))
        if cands: pred[str(cands[0].resolve())] = best_cls

# Assemble vectors (+ 'no_det' bucket)
NO_DET = NUM_CLASSES
label_names = CLASS_NAMES + ["no_det"]
y_true, y_pred, per_rows = [], [], []
for path, gt_cls in gt.items():
    p = pred.get(path, NO_DET)
    y_true.append(gt_cls); y_pred.append(p)
    per_rows.append({"image":path,"gt_id":gt_cls,"gt_name":CLASS_NAMES[gt_cls],
                     "pred_id":p,"pred_name":label_names[p]})

cm  = confusion_matrix(y_true, y_pred, labels=list(range(len(label_names))))
cmn = cm.astype(float) / (cm.sum(axis=1, keepdims=True) + 1e-9)

# classification_report (exclude 'no_det' from metrics)
rep = classification_report(
    y_true, [c if c < NUM_CLASSES else -1 for c in y_pred],
    labels=list(range(NUM_CLASSES)),
    target_names=CLASS_NAMES,
    output_dict=True,
    digits=4
)
rep_df = pd.DataFrame(rep).transpose()

EVAL_DIR = Path(f"runs/train/{RUN_NAME}/eval"); EVAL_DIR.mkdir(parents=True, exist_ok=True)
pd.DataFrame(per_rows).to_csv(EVAL_DIR/"per_image_predictions.csv", index=False)
pd.DataFrame(cm,  index=label_names, columns=label_names).to_csv(EVAL_DIR/"confusion_matrix_counts.csv")
pd.DataFrame(cmn, index=label_names, columns=label_names).to_csv(EVAL_DIR/"confusion_matrix_normalized.csv")
rep_df.to_csv(EVAL_DIR/"classification_report.csv", index=True)
print("Saved:\n ", EVAL_DIR/"per_image_predictions.csv",
      "\n ", EVAL_DIR/"confusion_matrix_counts.csv",
      "\n ", EVAL_DIR/"confusion_matrix_normalized.csv",
      "\n ", EVAL_DIR/"classification_report.csv")

# quick heatmaps
fig, axes = plt.subplots(1,2, figsize=(15,5))
im0 = axes[0].imshow(cm);  axes[0].set_title("Confusion (counts)")
axes[0].set_xlabel("Predicted"); axes[0].set_ylabel("True")
axes[0].set_xticks(range(len(label_names))); axes[0].set_yticks(range(len(label_names)))
axes[0].set_xticklabels(label_names, rotation=45, ha='right'); axes[0].set_yticklabels(label_names)
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        axes[0].text(j,i,int(cm[i,j]),ha='center',va='center',
                     color='white' if cm[i,j] > cm.max()/2 else 'black')
plt.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

im1 = axes[1].imshow(cmn); axes[1].set_title("Confusion (normalized)")
axes[1].set_xlabel("Predicted"); axes[1].set_ylabel("True")
axes[1].set_xticks(range(len(label_names))); axes[1].set_yticks(range(len(label_names)))
axes[1].set_xticklabels(label_names, rotation=45, ha='right'); axes[1].set_yticklabels(label_names)
for i in range(cmn.shape[0]):
    for j in range(cmn.shape[1]):
        axes[1].text(j,i,f"{cmn[i,j]:.2f}",ha='center',va='center',
                     color='white' if cmn[i,j] > cmn.max()/2 else 'black')
plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)
plt.tight_layout(); plt.show()


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

Saved:
  runs/train/y7_tiny_raw_clean3/eval/per_image_predictions.csv 
  runs/train/y7_tiny_raw_clean3/eval/confusion_matrix_counts.csv 
  runs/train/y7_tiny_raw_clean3/eval/confusion_matrix_normalized.csv 
  runs/train/y7_tiny_raw_clean3/eval/classification_report.csv


In [ ]:
# 5a) run detect on TEST (or VAL if no TEST)
import yaml, os
from pathlib import Path
with open(DATA_YAML, "r") as f:
    ds = yaml.safe_load(f)
TEST_IMAGES_DIR = Path(ds.get("test") or ds["val"])
BEST = Path(f"runs/train/{RUN_NAME}/weights/best.pt")

%cd /content/yolov7
!python detect.py --weights {str(BEST)} --img 640 --conf 0.001 \
  --source {str(TEST_IMAGES_DIR)} --save-txt --save-conf --name {RUN_NAME}_test

# 5b) build confusion matrix + report, save CSVs
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

CLASS_NAMES = ds["names"]; NUM_CLASSES = ds["nc"]
TEST_LABELS_DIR = Path(str(TEST_IMAGES_DIR).replace("/images/","/labels/"))
PRED_DIR = Path(f"runs/detect/{RUN_NAME}_test/labels")

# gt map
IMG_EXTS = {".jpg",".jpeg",".png",".bmp"}
gt = {}
for img in TEST_IMAGES_DIR.rglob("*"):
    if img.suffix.lower() in IMG_EXTS:
        lab = (TEST_LABELS_DIR / img.name).with_suffix(".txt")
        if lab.exists():
            parts = lab.read_text().strip().split()
            if parts: gt[str(img.resolve())] = int(parts[0])

# pred map (top-confidence per image)
pred = {}
if PRED_DIR.exists():
    for txt in PRED_DIR.rglob("*.txt"):
        lines = [ln.split() for ln in txt.read_text().splitlines() if ln.strip()]
        if lines:
            lines = [(int(p[0]), float(p[-1]) if len(p)>=6 else 0.0) for p in lines]
            top = max(lines, key=lambda t: t[1])[0]
            stem = txt.stem
            cands = list(TEST_IMAGES_DIR.rglob(stem + ".*"))
            if cands: pred[str(cands[0].resolve())] = top

# vectors (+ 'no_det' bucket)
NO_DET = NUM_CLASSES
label_names = CLASS_NAMES + ["no_det"]
y_true, y_pred = [], []
per_image_rows = []
for k, gt_cls in gt.items():
    p = pred.get(k, NO_DET)
    y_true.append(gt_cls); y_pred.append(p)
    per_image_rows.append({"image":k,"gt_id":gt_cls,"gt_name":CLASS_NAMES[gt_cls],
                           "pred_id":p,"pred_name":label_names[p]})

cm  = confusion_matrix(y_true, y_pred, labels=list(range(len(label_names))))
cmn = cm.astype(float) / (cm.sum(axis=1, keepdims=True) + 1e-9)
rep = classification_report(
    y_true, [x if x<NUM_CLASSES else -1 for x in y_pred],
    labels=list(range(NUM_CLASSES)), target_names=CLASS_NAMES, output_dict=True, digits=4
)
rep_df = pd.DataFrame(rep).transpose()

EVAL_DIR = Path(f"runs/train/{RUN_NAME}/eval"); EVAL_DIR.mkdir(parents=True, exist_ok=True)
pd.DataFrame(per_image_rows).to_csv(EVAL_DIR/"per_image_predictions.csv", index=False)
pd.DataFrame(cm,  index=label_names, columns=label_names).to_csv(EVAL_DIR/"confusion_matrix_counts.csv")
pd.DataFrame(cmn, index=label_names, columns=label_names).to_csv(EVAL_DIR/"confusion_matrix_normalized.csv")
rep_df.to_csv(EVAL_DIR/"classification_report.csv", index=True)
print("Saved:", EVAL_DIR/"classification_report.csv")

# quick plot
fig, axes = plt.subplots(1,2, figsize=(15,5))
im0 = axes[0].imshow(cm);  axes[0].set_title("Confusion (counts)")
axes[0].set_xlabel("Predicted"); axes[0].set_ylabel("True")
axes[0].set_xticks(range(len(label_names))); axes[0].set_yticks(range(len(label_names)))
axes[0].set_xticklabels(label_names, rotation=45, ha='right'); axes[0].set_yticklabels(label_names)
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        axes[0].text(j,i,int(cm[i,j]),ha='center',va='center',
                     color='white' if cm[i,j] > cm.max()/2 else 'black')
plt.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

im1 = axes[1].imshow(cmn); axes[1].set_title("Confusion (normalized)")
axes[1].set_xlabel("Predicted"); axes[1].set_ylabel("True")
axes[1].set_xticks(range(len(label_names))); axes[1].set_yticks(range(len(label_names)))
axes[1].set_xticklabels(label_names, rotation=45, ha='right'); axes[1].set_yticklabels(label_names)
for i in range(cmn.shape[0]):
    for j in range(cmn.shape[1]):
        axes[1].text(j,i,f"{cmn[i,j]:.2f}",ha='center',va='center',
                     color='white' if cmn[i,j] > cmn.max()/2 else 'black')
plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)
plt.tight_layout(); plt.show()


/content/yolov7
Namespace(weights=['runs/train/y7_tiny_raw_clean3/weights/best.pt'], source='/content/drive/MyDrive/DLRV/DATASET_YOLO_RAW_CLEAN/images/test', img_size=640, conf_thres=0.001, iou_thres=0.45, device='', view_img=False, save_txt=True, save_conf=True, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='y7_tiny_raw_clean3_test', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-128-ga207844 torch 2.8.0+cu126 CUDA:0 (Tesla T4, 15095.0625MB)

Traceback (most recent call last):
  File "/content/yolov7/detect.py", line 196, in <module>
    detect()
  File "/content/yolov7/detect.py", line 34, in detect
    model = attempt_load(weights, map_location=device)  # load FP32 model
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/yolov7/models/experimental.py", line 252, in attempt_load
    ckpt = torch.load(w, map_location=map_location)  # load
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

Saved: runs/train/y7_tiny_raw_clean3/eval/classification_report.csv
